In [1]:
#6->2->6

## Imports

In [2]:
import random

import pennylane as qml
from pennylane import numpy as np

from pennylane.optimize import AdamOptimizer

import torch
from torchvision import datasets, transforms

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tomut\anaconda3\envs\qhack2022\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:
import sys
sys.path.append("..") # Adds higher directory to python modules path

from qencode.initialize import setAB_amplitude, setAux, setEnt
from qencode.encoders import e3_enhance
from qencode.training_circuits import swap_t
from qencode.qubits_arrangement import QubitsArrangement

from qencode.utils.mnist import get_dataset

## Get Data

In [4]:
input_data = get_dataset(img_width=8, img_height=8, train=True)
print("Original data set size:", len(input_data))

# Select only the pictures with numbers 0 or 1. (jus to compare with literature)
filtered_data = [image for image in input_data if image[1] in [0, 1]]
input_data = filtered_data
print("Final data set szize:", len(input_data))

Original data set size: 60000
Final data set szize: 12665


## Training node

In [5]:
shots = 2500
nr_trash=4
nr_latent=2
nr_ent=0
nr_layers = 4

spec = QubitsArrangement(nr_trash, nr_latent, nr_swap=1, nr_ent=nr_ent, nr_aux=nr_trash+nr_ent)
print("Qubits:", spec.qubits)

#set up the device 
dev = qml.device("default.qubit", wires=spec.num_qubits)

Qubits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [6]:
@qml.qnode(dev)
def training_circuit_example(init_params, encoder_params, reinit_state, x):
    # Initialization
    setAB_amplitude(spec, init_params)
    setAux(spec, reinit_state)
    setEnt(spec, inputs=[1 / np.sqrt(2), 0, 0, 1 / np.sqrt(2)])

    #encoder 
    for params in encoder_params:
        e3_enhance(params, x, spec)

    #swap test 
    swap_t(spec)

    return [qml.probs(i) for i in spec.swap_qubits]

## Training parameters 

In [7]:
epochs = 2000
learning_rate = 0.0003
batch_size = 5
num_samples = 20

beta1 = 0.9
beta2 = 0.999
opt = AdamOptimizer(learning_rate, beta1=beta1, beta2=beta2)

In [8]:
def fid_func(output):
    # Implemented as the Fidelity Loss
    # output[0] because we take the probability that the state after the 
    # SWAP test is ket(0), like the reference state
    fidelity_loss = 1 / output[0]
    return fidelity_loss

In [9]:
def cost(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0][0],
                                          encoder_params=encoder_params,
                                          reinit_state=reinit_state, x=x[1])[0]
        f = fid_func(output)
        loss = loss + f
    return loss / len(X)

In [10]:
def fidelity(encoder_params, X):
    reinit_state = [0 for _ in range(2 ** len(spec.aux_qubits))]
    reinit_state[0] = 1.0
    loss = 0.0
    for x in X:
        output = training_circuit_example(init_params=x[0][0],
                                          encoder_params=encoder_params,
                                          reinit_state=reinit_state, x=x[1])[0]
        f = output[0]
        loss = loss + f
    return loss / len(X)

In [11]:
def iterate_batches(X, batch_size):
    X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in X]
    X2 = []
    for i in range(len(X1)):
        X2.append([X1[1], X[i][1]])
    X = X2
    random.shuffle(X)

    batch_list = []
    batch = []
    for x in X:
        if len(batch) < batch_size:
            batch.append(x)

        else:
            batch_list.append(batch)
            batch = []
    if len(batch) != 0:
        batch_list.append(batch)
    return batch_list

In [12]:
training_data = [input_data[i] for i in range(num_samples)]
test_data = [input_data[i] for i in range(num_samples,num_samples+num_samples)]

In [13]:
X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in training_data]
X2 = []
for i in range(len(X1)):
    X2.append([X1[1], training_data[i][1]])
X_training = X2

X1 = [torch.reshape(x[0], (1, 2 ** (len(spec.latent_qubits) + len(spec.trash_qubits)))) for x in test_data]
X2 = []
for i in range(len(X1)):
    X2.append([X1[1], training_data[i][1]])
X_tes = X2

In [14]:
# initialize random encoder parameters

nr_encod_qubits = len(spec.trash_qubits) + len(spec.latent_qubits)
nr_par_encoder = nr_layers * 2 * nr_encod_qubits + 2 * len(spec.trash_qubits)
encoder_params = np.random.uniform(size=(1, nr_par_encoder), requires_grad=True)

###  training

In [ ]:
loss_hist=[]
fid_hist=[]

loss_hist_test=[]
fid_hist_test=[]

for epoch in range(epochs):
    batches = iterate_batches(X=training_data, batch_size=batch_size)
    for xbatch in batches:
        encoder_params = opt.step(cost, encoder_params, X=xbatch)

        
    if epoch%5 == 0:
        
        loss_training = cost(encoder_params, X_training )
        fidel = fidelity(encoder_params, X_training )
        
        loss_hist.append(loss_training)
        fid_hist.append(fidel)
        print("Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_training, fidel))

        loss_test = cost(encoder_params, X_tes )
        fidel = fidelity(encoder_params, X_tes )
        loss_hist_test.append(loss_test)
        fid_hist_test.append(fidel)
        print("Test-Epoch:{} | Loss:{} | Fidelity:{}".format(epoch, loss_test, fidel))

C:\Users\tomut\anaconda3\envs\qhack2022\lib\site-packages\pennylane\math\multi_dispatch.py:63: UserWarning: Contains tensors of types {'torch', 'autograd'}; dispatch will prioritize TensorFlow and PyTorch over autograd. Consider replacing Autograd with vanilla NumPy.
  warnings.warn(


Epoch:0 | Loss:1.9715840327381713 | Fidelity:0.5072215533111285
Test-Epoch:0 | Loss:1.9584484595765161 | Fidelity:0.5106094979618462
Epoch:5 | Loss:1.967848055995158 | Fidelity:0.5081824549222912
Test-Epoch:5 | Loss:1.953516518468767 | Fidelity:0.511897684666687
Epoch:10 | Loss:1.9635621093322417 | Fidelity:0.5092889172893889
Test-Epoch:10 | Loss:1.9479557135661767 | Fidelity:0.5133587205685419
Epoch:15 | Loss:1.9587329947641343 | Fidelity:0.5105415548337555
Test-Epoch:15 | Loss:1.9418014190092787 | Fidelity:0.5149866888787928
Epoch:20 | Loss:1.9531930190657463 | Fidelity:0.5119864683124258
Test-Epoch:20 | Loss:1.9348366125136844 | Fidelity:0.5168434963324205
Epoch:25 | Loss:1.9469830236177361 | Fidelity:0.5136167635679303
Test-Epoch:25 | Loss:1.9271250912085993 | Fidelity:0.5189178433459507
Epoch:30 | Loss:1.9400167865999596 | Fidelity:0.5154595267815345
Test-Epoch:30 | Loss:1.9185541204102805 | Fidelity:0.5212471337425426
Epoch:35 | Loss:1.9322866484803771 | Fidelity:0.51752244348970

Test-Epoch:300 | Loss:1.5743643225201691 | Fidelity:0.6363615762248424
Epoch:305 | Loss:1.6063783051695275 | Fidelity:0.6232965705549888
Test-Epoch:305 | Loss:1.5717781575675063 | Fidelity:0.6373925146533959
Epoch:310 | Loss:1.6039444002395737 | Fidelity:0.6242280243684448
Test-Epoch:310 | Loss:1.5692475660408525 | Fidelity:0.6384071743978648
Epoch:315 | Loss:1.6015312720092836 | Fidelity:0.6251494491403445
Test-Epoch:315 | Loss:1.5668374036087538 | Fidelity:0.6393681056208426
Epoch:320 | Loss:1.5991757357328655 | Fidelity:0.6260518497360993
Test-Epoch:320 | Loss:1.564482911610634 | Fidelity:0.6403102443505747
Epoch:325 | Loss:1.5968751369360554 | Fidelity:0.6269360882343096
Test-Epoch:325 | Loss:1.5621798721314986 | Fidelity:0.6412351634978402
Epoch:330 | Loss:1.594642983681322 | Fidelity:0.6277990548959643
Test-Epoch:330 | Loss:1.5599023069595295 | Fidelity:0.6421571308565356
Epoch:335 | Loss:1.592428420655295 | Fidelity:0.6286541346531529
Test-Epoch:335 | Loss:1.557720739494631 | Fi

Epoch:605 | Loss:1.4995705594503141 | Fidelity:0.6678349002864395
Test-Epoch:605 | Loss:1.4702763131573537 | Fidelity:0.6813537926904872
Epoch:610 | Loss:1.498095534564629 | Fidelity:0.6685048317508823
Test-Epoch:610 | Loss:1.4690000975396833 | Fidelity:0.6819486845730177
Epoch:615 | Loss:1.4966070011324284 | Fidelity:0.6691838361117385
Test-Epoch:615 | Loss:1.467738559366188 | Fidelity:0.6825392336027725
Epoch:620 | Loss:1.4951245485022142 | Fidelity:0.6698563010817395
Test-Epoch:620 | Loss:1.4664574954699976 | Fidelity:0.6831343186814296
Epoch:625 | Loss:1.4936106169391976 | Fidelity:0.6705496098241968
Test-Epoch:625 | Loss:1.4652202441217843 | Fidelity:0.683714911306189
Epoch:630 | Loss:1.4920985020212192 | Fidelity:0.6712377006184334
Test-Epoch:630 | Loss:1.4639483342987092 | Fidelity:0.6843066019487237
Epoch:635 | Loss:1.4905676279328859 | Fidelity:0.6719356155355286
Test-Epoch:635 | Loss:1.4626826162358326 | Fidelity:0.6848960100592942
Epoch:640 | Loss:1.4890239002001613 | Fideli

Test-Epoch:905 | Loss:1.327453172421553 | Fidelity:0.7584855508356672
Epoch:910 | Loss:1.3280683253003525 | Fidelity:0.7582442795066167
Test-Epoch:910 | Loss:1.3253988850084364 | Fidelity:0.7596867025043734
Epoch:915 | Loss:1.326293479734221 | Fidelity:0.7592529315344174
Test-Epoch:915 | Loss:1.3234298159688038 | Fidelity:0.7608408519095123
Epoch:920 | Loss:1.324557365929251 | Fidelity:0.7602250280307898
Test-Epoch:920 | Loss:1.32154341533387 | Fidelity:0.7619321116937676
Epoch:925 | Loss:1.3228801505286805 | Fidelity:0.7611620971513282
Test-Epoch:925 | Loss:1.3197155434055015 | Fidelity:0.762987790351656
Epoch:930 | Loss:1.3212139118569204 | Fidelity:0.7620764161733695
Test-Epoch:930 | Loss:1.3179348721969013 | Fidelity:0.7639992059736318
Epoch:935 | Loss:1.3195994902669763 | Fidelity:0.7629617731374121
Test-Epoch:935 | Loss:1.3161879232375093 | Fidelity:0.7649916593409996
Epoch:940 | Loss:1.3180643134815178 | Fidelity:0.7638116724323106
Test-Epoch:940 | Loss:1.3144911414857012 | Fide

Epoch:1205 | Loss:1.2611712443959575 | Fidelity:0.7945800830320426
Test-Epoch:1205 | Loss:1.2546146668023406 | Fidelity:0.7989281771310506
Epoch:1210 | Loss:1.260214555222202 | Fidelity:0.7951239608234477
Test-Epoch:1210 | Loss:1.253585489341858 | Fidelity:0.7995184069096883
Epoch:1215 | Loss:1.2592684580633793 | Fidelity:0.7956639522499815
Test-Epoch:1215 | Loss:1.252565722760144 | Fidelity:0.8001054905414711
Epoch:1220 | Loss:1.2583263178377229 | Fidelity:0.7962035804216553
Test-Epoch:1220 | Loss:1.2515476640248204 | Fidelity:0.8006937010057043
Epoch:1225 | Loss:1.257364783682045 | Fidelity:0.796755427810768
Test-Epoch:1225 | Loss:1.2505000571048033 | Fidelity:0.8013011062453964
Epoch:1230 | Loss:1.256481054185897 | Fidelity:0.7972668821029322
Test-Epoch:1230 | Loss:1.249556686726279 | Fidelity:0.8018498795799118
Epoch:1235 | Loss:1.2555978980401428 | Fidelity:0.797779137295744
Test-Epoch:1235 | Loss:1.2486175716929167 | Fidelity:0.8023971260861472
Epoch:1240 | Loss:1.254683263554736

## Rezults

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist),label="train fid")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(fid_hist_test),label="test fid")


plt.legend()
plt.title("6-2-6:fidelity e3",)
plt.xlabel("epoch")
plt.ylabel("fid")

print("fidelity:",fid_hist[-1])

In [ ]:
fig = plt.figure()
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist),label="train loss")
plt.plot([x for x in range(0,len(loss_hist)*5,5)],np.array(loss_hist_test),label="test loss")


plt.legend()
plt.title("6-2-6:loss e3",)
plt.xlabel("epoch")
plt.ylabel("loss")

print("loss:",loss_hist[-1])

In [ ]:
name = "training_e3"

Circuit_prop={   "shots":shots, "nr_trash":nr_trash, "nr_latent":nr_latent, "nr_ent":nr_ent }
Training_param = { "num_samples" : num_samples,
                    "batch_size" : batch_size,
                    "nr_layers": nr_layers,
                    "epochs" :epochs,
                    "learning_rate" : learning_rate ,
                    "beta1" : beta1,
                    "beta2 ": beta2,
                     "optimizer":"Adam"}

performance={"loss_hist":loss_hist, "fid_hist":fid_hist,
             "loss_hist_test":loss_hist_test, "fid_hist_test":fid_hist_test,
             "encoder_params":encoder_params}

experiment_data={"Circuit_prop":Circuit_prop,
                "Training_param":Training_param,
                "performance:":performance,
                "Name":name}

# open file for writing
f = open(name+".txt","w")
f.write( str(experiment_data) )